In [ ]:
%matplotlib inline
from collections import OrderedDict
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import math
import os
from scipy.optimize import brentq
from sklearn.neighbors import KernelDensity
import xarray as xr

from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.kalman import KalmanFilter as KF
from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.common import Q_continuous_white_noise

from pumahu import Forwardmodel, ns_sampling, LakeData, get_data
from pumahu.visualise import trellis_plot, TrellisPlot, mcmc_heat_input, heat_vs_rsam, plot_qin_uks
from pumahu.uks import UnscentedKalmanSmoother
from pumahu.fd import fd

In [ ]:
from scipy import stats

In [ ]:
startdate = datetime(2003, 1, 16)
#startdate = datetime(2006, 6, 16)
enddate = datetime(2010, 1, 29)
#enddate = datetime(2008, 1, 1)
res_dir = './data'
if not os.path.isdir(res_dir):
    os.makedirs(res_dir)
mout_prior = get_data('./data/outflow_prior.npz')

In [ ]:
ld = LakeData(csvfile=get_data('data/data.csv'), enthalpy=3.0, windspeed=3.5,
              m_out=0, m_out_err=1.)

In [ ]:
xdf = ld.get_data(startdate, enddate, smoothing={'Mg': 2.6, 'T': 0.4, 'z': 0.5}, ignore_cache=True)

In [ ]:
if False:
    # set outflow data for dam break in March 2007
    xdf.loc['2007-03-23', 'm_out', 'val'] = 80
    xdf.loc['2007-03-23', 'm_out', 'std'] = 5.

In [ ]:
if True:
    xdf = xdf.interpolate_na(dim='dates')
else:
    xdf = xdf.dropna('dates', how='all')

In [ ]:
if True:
    xdf.loc['2007-02-19': '2007-03-24':, 'm_out', 'val'] = 30.

In [ ]:
P0 = OrderedDict(T=1e0, M=1e0, X=1e0, q_in=1e1,
                 m_in=1e1, m_out=1e1, h=1e-1, W=1e-1,
                 dqi=1e-2, dMi=1e-3, dMo=1e-3, dH=1e-3, 
                 dW=1e-3)
P0 = np.eye(len(P0))*list(P0.values())

Q = OrderedDict(T=1e0, M=1e2, X=1e-1, q_in=3e1,
                m_in=1e0, m_out=1e0, h=1e-3, W=1e-3,
                dqi=1e-3, dMi=0, dMo=0, dH=0, dW=0)
Q = np.eye(len(Q))*list(Q.values())
uks = UnscentedKalmanSmoother(data=xdf, Q=Q, P0=P0,
                              initvals={'qi': 0., 'm_in': 0., 'm_out': 20., 'X': 5.})
#uks = UnscentedKalmanSmoother(data=xdf,
#                              initvals={'qi': 0., 'm_in': 0., 'm_out': 20., 'X': 5.})
xds_uks = uks(smooth=True) 

In [ ]:
ft_in = '/home/behrya/workspace/pumahu/tests/data/LAKEOUTm.DAT'
qis, Ts, mis, dates = [], [], [], []
with open(ft_in, 'r') as fh:
    for i in range(4):
        fh.readline()
    while True:
        line = fh.readline()
        entries = line.split()
        if len(entries) < 18:
            break
        date = datetime(int(entries[0]),
                        int(entries[1]),
                        int(entries[2]))
        T = float(entries[3])
        qi = float(entries[5])
        mi = float(entries[7])
        qis.append(qi)
        Ts.append(T)
        mis.append(mi)
        dates.append(date)
        
ndates = len(dates)
parameters = ['q_in', 'm_in', 'T']
nparams = len(parameters)
exp = np.zeros((ndates, nparams, 2))*np.nan
m_in_idx = parameters.index('m_in')
exp[:, m_in_idx, 0] = mis
exp[:, m_in_idx, 1] = np.zeros(ndates)
q_in_idx = parameters.index('q_in')
exp[:, q_in_idx, 0] = qis
exp[:, q_in_idx, 1] = np.zeros(ndates)
T_idx = parameters.index('T')
exp[:, T_idx, 0] = Ts
exp[:, T_idx, 1] = np.zeros(ndates)      
res = xr.Dataset({'exp': (('dates', 'parameters', 'val_std'), exp)},
                 {'dates': dates,
                  'parameters': parameters,
                  'val_std': ['val', 'std']})

In [ ]:
trellis_plot(xds_uks, data2=res, dropzeros=False)

In [ ]:
xds_uks.exp.loc[:, 'q_in', 'val'].values

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
data = xds_uks.exp.loc[:, 'q_in', 'val'].values[:]
sigma = xds_uks.exp.loc[:, 'q_in', 'std'].values[:]
# Convert MW to TJ by multiplying with 86400 s
data *= 0.864
sigma *= 0.864
# Convert to kWh by deviding with 3.6e6 J
data /= 3.6
sigma /= 3.6
cum_mean = np.cumsum(data)
cum_std = np.sqrt(np.cumsum(sigma))
dates = pd.to_datetime(xds_uks.dates.values)
fig = make_subplots(specs=[[{"secondary_y": True}]])
#fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=cum_mean, mode='lines',
                         showlegend=False), secondary_y=False)
fig.add_trace(go.Scatter(name='Upper Bound',
                         x=dates,
                         y=cum_mean + 3*cum_std,
                         mode='lines',
                         marker=dict(color="#444"),
                         line=dict(width=0),
                         showlegend=False), secondary_y=False)
fig.add_trace(go.Scatter(name='Lower Bound',
                         x=dates,
                         y=cum_mean - 3*cum_std,
                         mode='lines',
                         marker=dict(color="#444"),
                         line=dict(width=0),
                         fillcolor='rgba(68, 68, 68, 0.3)',
                         fill='tonexty',
                         showlegend=False), secondary_y=False)
df_modis = pd.read_csv(get_data('data/41561_2021_705_MOESM2_ESM.csv'), usecols=[5, 6, 7, 8],
                        index_col=0, dayfirst=True, parse_dates=True)
df_modis = df_modis.loc[str(dates[0]):str(dates[-1])]
fig.add_trace(go.Scatter(x=df_modis.index, y=df_modis['Ruapehu_anomaly'].values), secondary_y=True)
fig.update_yaxes(title='$10^6 kWh$')
fig.update_layout(font_size=22)

In [ ]:
plot_wavelet_analysis(dates, data, nscales=256)

In [ ]:
data2.shape

In [ ]:
plt.specgram(data2, 100, 1, detrend='mean', noverlap=50)

In [ ]:
startdate2 = '1993-4-20'
enddate2 = '1994-7-4'
ld2 = LakeData(enthalpy=3.0, windspeed=3.5, m_out=0, m_out_err=10.)
xdf2 = ld2.get_data(startdate2, enddate2, smoothing={'Mg': 2.6, 'T': 0.4, 'z': 0.5}, ignore_cache=True)
xdf2 = xdf2.interpolate_na(dim='dates')

P0 = OrderedDict(T=1e0, M=1e0, X=1e0, q_in=1e1,
                 m_in=1e1, m_out=1e1, h=1e-1, W=1e-1,
                 dqi=1e-2, dMi=1e-3, dMo=1e-3, dH=1e-3, 
                 dW=1e-3)
P0 = np.eye(len(P0))*list(P0.values())
Q = OrderedDict(T=1e0, M=1e2, X=1e-1, q_in=3e1,
                m_in=1e0, m_out=1e0, h=1e-3, W=1e-3,
                dqi=1e-3, dMi=0, dMo=0, dH=0, dW=0)
Q = np.eye(len(Q))*list(Q.values())
uks = UnscentedKalmanSmoother(data=xdf2, Q=Q, P0=P0,
                              initvals={'qi': 0., 'm_in': 0., 'm_out': 20., 'X': 5.})
xds_uks = uks() 

In [ ]:
trellis_plot(xds_uks.loc[dict(dates=slice(startdate2, '1994-2-14'))], dropzeros=False)

In [ ]:
xds_uks.loc[dict(dates=slice(startdate2, '1994-4-1'))]